In [9]:
import re
from functions import *


In [8]:
file = "files\\Men\\en_ 2022-2024 MAG CoP.pdf"
# file = "files/Men/Reglement-Bovenbouw-versie-1.1.pdf"
page = loadPages(file, 161)
text= page[0].get_text()
print(text)


page range: [161]
A = 0,1
B = 0,2
C = 0,3
D = 0,4
E = 0,5
F = 0,6 G = 0,7 H = 0,8
Code MAG July 2022
155
50. Double salto arrière gr. par-dessus de la barre.
	
Double salto bwd. t. over the bar.
 	
Doble mortal atrás agr. por encima de la barra.
51. Double salto arrière gr. ou carpé avec 1/1 t. 
par- dessus la barre.
	
Doble salto bwd. t. with 1/1 t. over the bar.
 
Doble mortal atrás agr. con 1/1 g. por encima 
de la barra.
58. 
Double salto arrière tendu avec 1/1 t. par-dessus 
la barre.
	
Double salto bwd. str. w. 1/1 t. over the bar.
 	
Doble mortal ext. con 1/1 g. por encima de la b.
(Hayden)
57. Double salto arrière tendu par-dessus la barre.
	
Double salto bwd. str over the bar. 
  
Doble mortal ext. por encima de la barra.
52. Double salto arrière gr. avec 2/1 t. par- dessus 
la barre.
	
Double salto bwd. t. with 2/1 t. over the bar
  
Doble mortal atrás agr. con 2/1 g. por encima 
de la barra.
53. 
60. 
Triple salto arrière gr. avec 1/1 t.
	
Triple salto bwd. t. with 1/1 t.
 


In [7]:

for line in text.split("\n"):
    print(line)

Verboden voor jeugd en junioren turners 
 
Versie 1.1 (Oktober-2022 ) 
 
Bovenbouw TH 2022  
98 
A = 0.1 
B = 0.2 
C = 0.3 
D = 0.4 
E = 0.5 
F = 0.6 G = 0.7 H = 0.8 
EG IV Afsprongen 
1. Salto vw gehoekt of gestrekt, 
ook met ½ draai 
2. Salto vw gehoekt of gestrekt 
met 1/1 draai 
3. Salto vw gehoekt of gestrekt 
met 3/2 draai 
4. Salto vw gehoekt of gestrekt 
met 2/1 draai 
5.  
6.  
7.  
8.  
9. Dubbel salto vw gehurkt met 
of zonder ½ draai 
10. Dubbel salto vw met 1/1 of 
3/2 draai 
11.  
12.  
13.  
14.  
15.  
16. Dubbel salto vw gehoekt 
met of zonder ½ draai 
17. Dubbel salto vw gehoekt 
met 1/1 of 3/2 draai 
18.  
19.  
20.  
21.  
22.  
23.  
24.  



In [38]:
y = text.replace("\n", "|").replace("\t", "").replace("||", "|")

In [39]:
print([y])

['A = 0,1|B = 0,2|C = 0,3|D = 0,4|E = 0,5|F = 0,6 G = 0,7 H = 0,8|Code MAG July 2022|42|1. De l’appui renvs. s’abaisser à l’équerre ou à |l’équerre écartée (2 s.).|From hdst. lower to L-sit or strad. L-sit (2 s.).|Desde ap. inv.., bajar a ángulo o ángulo p. ab. (2 s.).|2. Équerre, jambes levées à la vertical (2 s.).|V-sit (2 s.).| |Ángulo, piernas verticales (2 s.).|3. Équerre, jambes à l’horizontal (2 s.).|V-sit with legs horiz. (2 s.).| |Ángulo piernas horizontales (2 s.).|7. De la station jambes éc., appui renv. (2 s.).|From straddle stand, press to hdst. (2 s.).| |Desde el ap. piernas ab., apoyo invertido (2 s.).|8. S’élever corps fléchi, bras tendus ou corps |tendu, bras fléchi à l’appui renv. (2 s.).|Piked body, str. arm or str. body, bent arm press |to hdst. with legs together (2 s.).| |Elevarse cuerpo flex., brazos ext. o cuerpo ext., |brazos flex. al apoyo invertido (2 s.).|9. |l’Équerre jambes vert.(2 s.) et s’élever à l’appui |renv. (2 s.). Aussi jambes écartées dans la phas

In [7]:

pattern = r"(\d+)\.\s+(.*?)\s*(?=\|\d+\.|\Z)"
for res in re.findall(pattern, text.replace("\t", "").replace("\n", "|").replace("||", "|")):
    print(res)
    # if res[0] == "110":
    #     break

('1', 'De l’appui renvs. s’abaisser à l’équerre ou à |l’équerre écartée (2 s.).|From hdst. lower to L-sit or strad. L-sit (2 s.).|Desde ap. inv.., bajar a ángulo o ángulo p. ab. (2 s.).')
('2', 'Équerre, jambes levées à la vertical (2 s.).|V-sit (2 s.).| |Ángulo, piernas verticales (2 s.).')
('3', 'Équerre, jambes à l’horizontal (2 s.).|V-sit with legs horiz. (2 s.).| |Ángulo piernas horizontales (2 s.).')
('7', 'De la station jambes éc., appui renv. (2 s.).|From straddle stand, press to hdst. (2 s.).| |Desde el ap. piernas ab., apoyo invertido (2 s.).')
('8', 'S’élever corps fléchi, bras tendus ou corps |tendu, bras fléchi à l’appui renv. (2 s.).|Piked body, str. arm or str. body, bent arm press |to hdst. with legs together (2 s.).| |Elevarse cuerpo flex., brazos ext. o cuerpo ext., |brazos flex. al apoyo invertido (2 s.).')
('9', '|l’Équerre jambes vert.(2 s.) et s’élever à l’appui |renv. (2 s.). Aussi jambes écartées dans la phase |finale.| V-sit (2 s) and press to hdst. (2 s.). Als

In [59]:
for x in re.split(r"\|(?=[A-Z])", s):
    print("t", x)

t Rondade, ½ t. et renversement av. et double |salto avant groupé.	
t Round off, ½ t. a. hdspr. fwd. and double salto |fwd. tucked.
t Rondada, ½ g. paloma doble mortal ad. agr.


In [38]:
print(re.split(r"\|(?=[A-Z|\s|\d|_])", '_Renvs._ av._et_salto_av._gr._avec_1/1_t._(Cuervo_|gr._a._½_t.).| Hdspr. fwd. and salto fwd. t. w. 1/1 t. (Cuervo t. |w. ½ t.).|_ Paloma_ mortal_ad._agr._con_1/1_g._(Cuervo_agr.__|con._½_g.).'))

['_Renvs._ av._et_salto_av._gr._avec_1/1_t._(Cuervo_|gr._a._½_t.).', ' Hdspr. fwd. and salto fwd. t. w. 1/1 t. (Cuervo t. |w. ½ t.).', '_ Paloma_ mortal_ad._agr._con_1/1_g._(Cuervo_agr.__|con._½_g.).']


In [14]:
for r in res.split("|"):
    print(r, langdetect.detect_langs(r) )



Appui   facial  horizontal.     (2      s.). [ro:0.9999938676465245]
 Support scale (2 s.). [fr:0.8571386350232229, it:0.1428585880455012]
     Apoyo  facial  horizontal      (2      s.). [es:0.9999951456983701]


In [34]:
loadConfig("element_grouping.yaml")["apparatuses"]["men"]["vault"]["type"]

Loading config file


['handstandoverslag / yamashita',
 'tsukahara',
 'arabier flik-flak / yurchenko',
 'streksprong']

In [2]:
print(" Hecht strad., hecht or hecht w. \u00bd t.".strip())


Hecht strad., hecht or hecht w. ½ t.


In [39]:
vault = loadJson("../data/elements/men/en_elements.json")
vault_nrs = [x["number"] for x in vault["vault"]]
values = loadConfig("files/Men/vault_values.yaml")
missing = set(values.keys()) - set(vault_nrs)
print(sorted(missing))



Loading config file
['3.309', '4.415', '4.449', '4.455', '4.456', '4.457', '4.458', '4.459', '4.460', '4.461', '4.462', '4.467', '4.468', '4.469', '4.470', '4.471', '4.472']
